## Importing libraries and installing packages

In [33]:
!pip install langchain --quiet
!pip install apify-client --quiet
!pip install openai --quiet
!pip install tiktoken --quiet

In [34]:
from langchain.document_loaders.base import Document
from langchain.indexes import VectorstoreIndexCreator
from langchain.utilities import ApifyWrapper

In [35]:
import sqlalchemy
db_connection = sqlalchemy.create_engine(connection_url).connect()

## OpenAI function

In [36]:
import openai
import json

# models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"
openai.api_key = "open-api-key-here"

In [37]:
def get_completion(prompt,): 
    messages = [
        {"role": "system", "content": 
             """You are a helpful assistant.
            Given information regarding a person and their background, generate a very highly personalized email
            for them with the mission of convincing them to join SingleStore as a customer.
            """},
        {"role": "user", "content": f"This is the information about this person:'{prompt}'"}
    ]
    response = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

## Import APIFY

In [38]:
import os
os.environ["OPENAI_API_KEY"] = "open-api-key-here"
os.environ["APIFY_API_TOKEN"] = "apify-api-token-here"

apify = ApifyWrapper()

from apify_client import ApifyClient
client = ApifyClient(os.environ["APIFY_API_TOKEN"])

import requests

## Create Database & Add Information

In [39]:
%%sql
CREATE DATABASE webinarattendees;
USE webinarattendees;
CREATE TABLE openAIWebinar7_16_23 (
    `firstName` text CHARACTER SET utf8 COLLATE utf8_general_ci,
    `email` text CHARACTER SET utf8 COLLATE utf8_general_ci,
    `linkedin` text CHARACTER SET utf8 COLLATE utf8_general_ci
    );
    
INSERT INTO openAIWebinar7_16_23 (firstName, email) VALUES 
    ("Trisha", "trisha.maturi@gmail.com"),
    ("Madhukar", "madhukar@singlestore.com");

## The scraping and analyzing:

In [40]:
# The resulting list that has all the final messages/results
output_list = [];

In [41]:
query = "SELECT firstName, email FROM webinarattendees.openAIWebinar7_16_23;"

In [42]:
for row in db_connection.execute(query):
    input = row[0] + " " + row[1]
    # Get the linkedin profile from the email
    run_input = { "queries": f"""{input}""" }
    
    # Run the actor
    run = client.actor("anchor/linkedin-people-finder").call(run_input=run_input)
    
    # Update it in the table
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        linkurl = item['linkedinUrl'];
        queryupdate = "UPDATE webinarattendees.openAIWebinar7_16_23 SET linkedin = %s WHERE email = %s;"
        db_connection.execute(queryupdate, (linkurl, row[1]))
            
    # Run the second actor and get response
    username = (linkurl).rsplit('/', 1)[1]
    payload = {'api_key': 'scraping-dog-api-token-here', 'type':'profile', 'linkId': username}
    resp = requests.get('https://api.scrapingdog.com/linkedin', params=payload)
    response = resp.json()
    x = json.dumps(response)
    info = json.loads(x)[0]
    
    # Get specific information about the user to pass into openAI
    background = "Name: " + info['fullName'];
    background += ", Headline: " + info['headline'];
    background += ", Location: " + info['location'];
    
    expcount = len(info['experience'])
    if expcount != 0:
        background += ". Experiences: "
        for i in range(expcount):
            if (i > 7):
                break
            background += info['experience'][i]['position'] + " at " + info['experience'][i]['company_name'] + ", "
            
    # Call the openAI Function to get the personalized message
    message = get_completion(background)
    
    # Add it to the list
    output_list.append([
        row[1], message
    ]);

In [169]:
print(output_list[1][0])

trisha.maturi@gmail.com


In [171]:
print(output_list[1][1])

Subject: Join SingleStore and Revolutionize Your Data Management, Trisha!

Dear Trisha Maturi,

I hope this email finds you well. My name is [Your Name], and I am reaching out to you on behalf of SingleStore, the leading modern data platform. After reviewing your impressive background and experiences, I am convinced that you would greatly benefit from our innovative solutions.

As the Head of CS at the University of Michigan and being located in the technology hub of San Jose, California, you are undoubtedly aware of the challenges and opportunities that arise in the world of data management. At SingleStore, we specialize in providing a comprehensive and scalable platform that empowers organizations to unlock the full potential of their data.

Your research experience with Professor Eytan Adar at the University of Michigan demonstrates your dedication to exploring cutting-edge technologies. SingleStore's advanced capabilities, such as real-time analytics, machine learning, and AI, alig

In [43]:
print(output_list[0][0])

madhukar@singlestore.com


In [44]:
print(output_list[0][1])

Subject: Join SingleStore and Unleash Your Growth Potential, Madhukar Kumar!

Dear Madhukar Kumar,

I hope this email finds you in good health and high spirits. My name is [Your Name], and I am reaching out to you on behalf of SingleStore, where you currently hold the esteemed position of Chief Marketing Officer. I have been following your remarkable journey as a CMO, speaker, author, and growth marketer, and I am truly impressed by your expertise and achievements.

As a fellow growth-driven professional, I understand the challenges and opportunities that come with scaling businesses in today's fast-paced digital landscape. That's why I am excited to introduce you to SingleStore, a cutting-edge platform that empowers companies to unlock their full growth potential through real-time analytics and data-driven decision-making.

Located in the heart of the San Francisco Bay Area, SingleStore is revolutionizing the way businesses handle their data. Our platform combines the power of traditi